# Description

### This script labels each frame with several parameters that are usefull for setting the values at freezing-detection script. It is based on DLC output too and just like the freezing-detection script, a folder containing csv&pickle (only) DLC outputs for each file must be used as path.

#### Please mind that the videos you choose here must correspond to your DLC output file. This means that if you have set a different starting point in the TrimFrames() of the freezing-detection script or in the DLC itself, the values that will be pasted to each frame won't correspond with the real values anymore. If you want to use a trimmed output file that doesn't start from frame 0, you'll have to trimm your selected video to the exact right point in terms of frames for that.
#### The interpolated values are indicated with a black instead of a yellow colour. These values have been interpolated according to the uncertainty range that you set in Script 1.

#### When renaming a file to get rid of the DLCResnet.... extension, make sure you include a _ before the meta.pickle. That is, _meta.pickle instead of meta_pickle.

#### Remember that the nose is used for detection of False Positives, therefore, even if the values are present in each frame, we are only interested in the nose values during the freezing ranges, as dictated by the main freezing script.

In [4]:
import pandas as pd
import numpy as np
import datetime
import sys
import copy
import datetime
import json
import itertools
import cv2
import plotly.express as px
import regex as re
from os import listdir
from os.path import isfile, join
from collections import defaultdict 
from itertools import cycle, islice 
from tkinter import *
from tkinter import filedialog
from tkinter.filedialog import askdirectory
from tkinter import messagebox
import pickle
import os

In [5]:
def SelectDLCOutputDirectory ():
    def select_specific_directory():
        global pickle_directory                                                                          # practically, only pickle needed, csv in the variable names is now obsolete
        pickle_directory = filedialog.askdirectory(title='Select DLC output folder (only csv&pickle)')
        root.destroy()
    root = Tk()
    root.title ('Select folder with DLC output (only csv&pickle)')
    root.resizable(True,True)
    root.geometry('400x150')
    DirButton = Button(root,text='Locate folder', command=select_specific_directory).place(x=150,y=50)
    root.mainloop()

def SelectVideoDirectory ():
    def select_spec_video_dir():
        global video_dir
        video_dir = filedialog.askdirectory(title='Video location')
        iroot.destroy()
    iroot = Tk()
    iroot.title ('Select folder with existent videos')
    iroot.resizable(True,True)
    iroot.geometry('400x150')
    DirButton = Button(iroot,text='Locate folder', command=select_spec_video_dir).place(x=150,y=50)
    iroot.mainloop()
    

def SelectSaveDirectory ():
    def select_spec_save_dir():
        global save_dir
        save_dir = filedialog.askdirectory(title='Save location')
        sroot.destroy()
    sroot = Tk()
    sroot.title ('Select folder to save new videos')
    sroot.resizable(True,True)
    sroot.geometry('400x150')
    DirButton = Button(sroot,text='Locate folder ', command=select_spec_save_dir).place(x=150,y=50)
    sroot.mainloop()

In [6]:
def UserSelections() :
    """Takes user's input for selection of bodyparts. betwears and nose is included by default"""

    selections =['betwears','nose']

    ask_for_second_part = input ('Do you want to include another body part apart from betwears and nose ? \n')
    if ask_for_second_part.upper() == 'YES' :
        wrong_part,which_part = True,True
        while wrong_part == True and which_part!='q' :
            which_part = input (' \n Which part do you want to include? Type left or right \n')
            if which_part.upper() in ['LEFT','RIGHT']:
                selections.append(which_part.lower())
                break
            else :
                if which_part !='q' :
                    print('\n You have typed something wrong. Please type left or right. Otherwise type q to exit')
                else :
                    print ('\n You have stopped the code.')
                    return 
    return selections

In [7]:
def RenameDataframeColumns() :
    """Renames the columns of the Dataframe with DLC coordinates (output of DLC analysis script).
       Column DLC_resnet50_Trigger_0005_008Aug30shuffle1_500000 will become betwearsx , column DLC_resnet50_Trigger_0005_008Aug30shuffle1_500000.1 will become
       betwearsy and DLC_resnet50_Trigger_0005_008Aug30shuffle1_500000.2 will become betwearslikelihood and so on"""
    new_cols = pickle_df['csv'].loc['bodyparts'].values.tolist()
    axis_inf = pickle_df['csv'].loc['coords'].values.tolist()
    for coord_index in range(len(new_cols)):
        coord= axis_inf[coord_index]
        new_cols[coord_index] = new_cols[coord_index]+coord
    pickle_df['csv'].columns = new_cols


In [8]:
def CheckIfSecondBodyPartIncluded():
    global selected_parts
    if len (selected_parts) == 3 :
        return selected_parts[2]                                                                                          # if there is a third element. It is a user input of 2nd bodypart. 
    else:
        return False


In [9]:
def AssignFutureFramesAndDenominators(pdata, sec_bdpart = False):
    """Input pdata : the extended df that also has the denominators key with future frames and denominator values info
        Output : list with values corresponding to denominators and future frames """
    
    
     #  - - - - - - - Assigning values for betwears avgd_diff calculation - - - - - - - - -
    btw_ff_denom = pdata['denominators']['betwears']['denominator_ff']                                                    # denominator for betwears (for calculation of avgd_diff ) based on freezing analysis script
    ff_betw      = pdata['denominators']['betwears']['ff']                                                                # number of future frames taken (for calculation of avgd_diff ) based on freezing analysis script                                               
    #  - - - - - - - Assigning values for nose avgd_diff calculation - - - - - - - - -
    nose_ff_denom = pdata['denominators']['nose']['denominator_ff']
    ff_nose       = pdata['denominators']['nose']['ff']
    #  - - - - - - - Assigning values for 2nd bodypart avgd_diff calculation - - - - - - - - -
    denominators_and_fframes= [btw_ff_denom,ff_betw,nose_ff_denom,ff_nose]                                                # first assign these 4 which are standard (irrespective of 2nd bodypart)
    
    if sec_bdpart != False:
        secbd_ff_denom = pdata['denominators'][f'{sec_bdpart}']['denominator_ff']
        ff_secbd       = pdata['denominators'][f'{sec_bdpart}']['ff']
    else :
        secbd_ff_denom = False                                                                                            # assign elements to list to avoid handling errors when callin function
        ff_secbd      = False     
    denominators_and_fframes.extend([secbd_ff_denom,ff_secbd])                                                            
    
    return denominators_and_fframes

    

In [10]:
def InitiateFrameVariables(vname,pkldata):
    vid_path = video_dir+'/'+vname
    vid_capture = cv2.VideoCapture(vid_path)
    frame_width, frame_height = int(vid_capture.get(3)),int(vid_capture.get(4))
    frame_size = (frame_width,frame_height)
    fps = pkldata['fps']
    font = cv2.FONT_HERSHEY_SIMPLEX
    output = cv2.VideoWriter(f'{save_dir}/{vname[:-4]}.avi', cv2.VideoWriter_fourcc('M','J','P','G'), fps, frame_size)
    # output = cv2.VideoWriter(f'{save_dir}/{vname[:-4]}.mp4', cv2.VideoWriter_fourcc('m','p','4','v'), fps, frame_size)   #for mp4 instead of avi


    return vid_capture,font,output

In [11]:
def InitiateCoordLabelVariables(pkldata,bdpart_opt=False):
    max_coord = pkldata['csv'].index[-1]                                                                                          # last row with a coordinate at the series 
    coor =  pkldata['csv']                                                                                                        # for easy assigning
    orange =  (0, 0, 0)
    yellow =  (0,255,255)
    interpolated_BTW, interpolated_NOSE = pkldata['interpolated_frames']['betwears'], pkldata['interpolated_frames']['nose']      # list with frames the values of which were interpolated
    
    
    if bdpart_opt != False:                                                                                                       # False only in the case the user opted for not second bodypart
        try :
            interpolated_SECBD = pkldata['interpolated_frames'][f'{bdpart_opt}']
        except KeyError :
            print (f'The selected bodypart {bdpart_opt} could not be loaded from your pickle output')
            interpolated_SECBD = []    
    else:
        interpolated_SECBD = False

    
    return max_coord,coor,orange,yellow,interpolated_BTW,interpolated_NOSE,interpolated_SECBD                                                                                        # create empty list to avoid error handling
    

In [12]:
def LabelPositions (width,height, upORlow, propor, leeway): 
    """Input width : int     ->the width of the video resolution 
             height: int     -> the height of the video resulution
             upORlow: string -> either u or l, for upper or lower respectively. Defines whether labels will be up or down
             propor :int     -> proportion of the upper or lower part where labels will be placed 
             leeway :int     -> highest (for u selection) and lowest (for l selection) pixel where the betwears (for u selection) or nose (for l selection) will be.

        Output : a merged list that contains first the width starting points where labels will appear and then starting points of height where labels will appear
             """
    lbl_dist_w = width/5                                                                                                           # get how much 1/5 of the overall width is
    width_output = [(int(round(lbl_dist_w*x))) for x in range(1,5)]                                                                # meaning lbl_dist_w*1, lbl_dist_w*2, lbl_dist_w*3 and lbl_dist_w*4

    if upORlow == 'u':
        height_output  = list(np.linspace(leeway,propor*height/100,num=3,dtype=int))                                               # 3 variables are assigned values from a list that contains 3 elements. This is a list with the starting points for the y axis (height) that will be used for the positioning of each body part in your video
    elif upORlow =='l' :
        leeway = height - leeway                                                                                                   # the 'lowest' where the labels of the nose will appear
        top_part = (100-propor) * height / 100                                                                                     # for instance, if proportion was 10, that would mean that the labels will start at 90% of the height of the video (instead of 10%, that is higher, if the ul selection was upper)
        height_output  = list(np.linspace(top_part,leeway,num=3,dtype=int))                                                        # first element is betwears height, second element the secnd bodypart height and third element noise height
    else :
        print ('Wrong input for placing labels on upper or lower part (variable named ul)./n The program will terminate')
        sys.exit(0)
    
    combined = width_output+height_output
    return combined


In [13]:
def FreezingAndTemporaryLabelsPositions (upORdown,ycoords,likecoords,locdiff_coords,noheight,betheight):
    """ Input upORdown       : string can be either 'u' or 'l' for upper or lower 
              ycoords        : int starting point of a label for y coords with respect to width
              likecoords     : int starting point of a label for p coords with respect to width
              locdiff_coords : int starting point of a label for loc_diff with respect to width
              noheight       : int starting point of a label for nose with respect to height
              betheight      : int starting point of a label for betwears with respect to height
        Output freezing_label_position               : int starting point or freezing label for between ears
               freezing_label_position_second_bdpart : int starting point or freezing label for second bodypart
               shrt_lbls_betw_positions              : int starting point for overall occurences of freezing temporary label for between ears
               shrt_lbls_secbd_positions             : int starting point for overall occurences of freezing temporary label for second bodypart
        """
    if upORdown == 'u':
        pixels_below_or_above = 30                                                                                                                # N of pixels under nose labels where freezing labels will show with respect to height
        plus_pixels_second_bdpart = 20
        reference_point = noheight                                           
    elif upORdown == 'l':
         pixels_below_or_above = - 30                                                                                                             # N of pixels above betwears labels where freezing labels will show with respect to height
         plus_pixels_second_bdpart = -20
         reference_point = betheight
    else :
        print('Wrong input for the variable ul that defines whether labels will be on top or at the bottom. This program will exit now')
        sys.exit(0)


    freezinglabel_pos = (round((locdiff_coords+likecoords)/2), reference_point + pixels_below_or_above)                                      # The freezing label will be in between p and loc_diff with respect to width and 30 pixels below the starting point of nose labels
    freezinglabel_pos_second_bdpart = freezinglabel_pos                                                                                      # you can change it if you want to avoid overlap of labels
    shrt_lbls_betw_pos = (round((likecoords+ycoords)/2), reference_point + pixels_below_or_above)                                            # The label indicating overall occurences of betwears detected freezing will be in between y and loc_diff with respect to width and 15 pixels below the starting point of nose labels
    shrt_lbls_secbd_pos = (round((likecoords+ycoords)/2), reference_point + pixels_below_or_above +plus_pixels_second_bdpart)                # The label indicating overall occurences of second bodypart detected freezing will be in between y and loc_diff with respect to width and 30 pixels below the starting point of nose labels

    return freezinglabel_pos,freezinglabel_pos_second_bdpart,shrt_lbls_betw_pos,shrt_lbls_secbd_pos

In [14]:
def InsertLabels (vname,pkldata,bdpart_opt=False):
    """Input vname  : str corresponding to a video's name
       pkldata      : an extended dict with many keys, as generated in the script for DLC analysis.
       bdpart_opt   : either False or a string with a 2ndry bodypart

       Important information : We are calculating avged_diff dividing by 4 but in fact the right number is 5.
       We have overal 5 past + 1 current frame observation, resulting in 5 differences (x2-1, x3-x2 etc). So 5 is the right
       value to divide with and not 4. However, the mistake was made in the main freezing code, hence we keep this as a default for betwears and 2nd bodypart
       and as 5 for nose"""
    
    
    # - - - - - - - - - - - - - - - Initiation of Variables - - - - - - - - - - - - - - -
    global selected_parts
    vid_capture,font,output = InitiateFrameVariables(vname,pkldata) 
    max_coord,coor,orange,yellow,interpolated_BTW, interpolated_NOSE, interpolated_SECBD = InitiateCoordLabelVariables(pkldata,bdpart_opt)
    BTW_ff_denom,ff_BETW,NOSE_ff_denom,ff_NOSE,SECBD_ff_denom,ff_SECBD = AssignFutureFramesAndDenominators (pkldata,bdpart_opt)                            # denom means denominators, ff means future frames. Future frames is the number of future frames we tale into account for calculating avged_diff. Denominator is the number we divide the sum of differences between these frames (see freezing analysis script)
    btw_msg = f'{len(pkldata["finals"])} betwears freez instances'                                                                                         # n of times of freezing based on betwears
    sec_msg = f'{len(pkldata["sec_finals"])} 2nd_bdpart instances(might overlap)'                                                                          # n of times of freezing based on second bdpart
    print(f'\n file will be saved at {save_dir}/{vname[:-4]}.avi')  
    y_ax_coords,likel_coords,loc_diff_coords,avg_diff_coords, betwears_height,sec_bd_height,nose_height = LabelPositions(vw,vh,ul,lbl_h_prop,height_start) # First 5 variables are the starting points of labels for width. Last 3 are the starting points for height
    freezing_label_position, freezing_label_position_second_bdpart, shrt_lbls_betw_positions, shrt_lbls_secbd_positions = FreezingAndTemporaryLabelsPositions(ul,y_ax_coords,likel_coords,loc_diff_coords,nose_height,betwears_height)    # Define positions for labels of freezing and temporary labels indicating overall occurences of freezing in the start of the video
    
    counter = -1


    # - - - - - - - - - - - - - - - Start calculating loc & avged_diff and writing to frame - - - - - - - - - - - - - - -

    while (vid_capture.isOpened()):
        counter = counter + 1
        boole, frame = vid_capture.read()        
                          
        if boole == True:
            frame_indicator = 'frame:' + str(counter)               
            frame = WritetoFrame(frame,frame_indicator,(avg_diff_coords,nose_height+30),font,labelsize,yellow,1,cv2.LINE_4)       # display n of frame, on the same width where avged_diff starts and 30 pixels lower than avged_diff for the nose is. You can do labelsize+N if you want larger frame labels than the rest

            # - - - - - - Labels for betwears- - - - - - 
            if counter <= max_coord-15:
                x_axis_indicator = 'betwears x: ' + str(round(coor['betwearsx'][counter],3))                                   
                y_axis_indicator = 'y: ' + str(round(coor['betwearsy'][counter],3))
                l_axis_indicator = 'p: ' + str(round(coor['betwearslikelihood'][counter],8))  
                
                curr_x_row, curr_y_row  = coor['betwearsx'][counter], coor['betwearsy'][counter]                                  # code to calculate the loc_diff
                next_x_row, next_y_row  = coor['betwearsx'][counter+1], coor['betwearsy'][counter+1]
                total_diff_both_axes = abs (curr_x_row-next_x_row) + abs (curr_y_row-next_y_row)
                loc_diff_indicator = 'loc_diff: ' + str(round(total_diff_both_axes,3))

                next_N_observ_x = coor['betwearsx'].loc[counter+1:counter+ff_BETW]                                                # observations from subsequent frames' coordinates to calculate avged_diff.In the main freezing code we use dataframes, which have different indexing. Hence the counter+7 instead of +6. Note that counter+2 doesnt exist here. series[counter] is not the same with the first element of series [counter:counter+something]. Series[counter] calls the row with counter whereas Series[counter:counter+something] starts giving values 0,1,2 to row 0,1,2 and so on.
                next_N_observ_y = coor['betwearsy'].loc[counter+1:counter+ff_BETW]
                consec_frames_x_diff, consec_frames_y_diff = np.ediff1d (next_N_observ_x), np.ediff1d(next_N_observ_y)
                total_x_y_diff = (abs(sum(consec_frames_x_diff)) /BTW_ff_denom) \
                               + (abs(sum(consec_frames_y_diff)) /BTW_ff_denom) 
                avged_diff_indicator = 'avged_diff: ' + str(round(total_x_y_diff,3))
                
                if counter in interpolated_BTW:                                                                                   # if the value of betwears is interpolated , will be different color
                    color = orange
                else :
                    color = yellow

                frame= WritetoFrame(frame,x_axis_indicator,(1,betwears_height),font,labelsize,color,1,cv2.LINE_4)
                frame= WritetoFrame(frame,y_axis_indicator,(y_ax_coords,betwears_height),font,labelsize,color,1,cv2.LINE_4)
                frame= WritetoFrame(frame,l_axis_indicator,(likel_coords,betwears_height),font,labelsize,color,1,cv2.LINE_4)
                frame= WritetoFrame(frame,loc_diff_indicator,(loc_diff_coords,betwears_height),font,labelsize,color,1,cv2.LINE_4)
                frame= WritetoFrame(frame,avged_diff_indicator,(avg_diff_coords,betwears_height),font,labelsize,color,1,cv2.LINE_4)
                
                # - - - - - - Labels for nose - - - - - -             
                nose_x_axis_indicator = 'nose     x: ' + str(round(coor['nosex'][counter],3))
                nose_y_axis_indicator = 'y: ' + str(round(coor['nosey'][counter],3))
                nose_l_axis_indicator = 'p: ' + str(round(coor['noselikelihood'][counter],8))  
                 
                nose_curr_x_row, nose_curr_y_row  = coor['nosex'][counter], coor['nosey'][counter]                                 #code to calculate the loc_diff
                nose_next_x_row, nose_next_y_row  = coor['nosex'][counter+1], coor['nosey'][counter+1]
                nose_total_diff_both_axes = abs (nose_curr_x_row-nose_next_x_row) + abs (nose_curr_y_row-nose_next_y_row)
                nose_loc_diff_indicator = 'loc_diff: ' + str(round(nose_total_diff_both_axes,3))
                
                nose_next_N_observ_x  = coor['nosex'].loc[counter+1:counter+ff_NOSE]    
                nose_next_N_observ_y  = coor['nosey'].loc[counter+1:counter+ff_NOSE]                                                              
                nose_consec_frames_x_diff, nose_consec_frames_y_diff = np.ediff1d (nose_next_N_observ_x), \
                np.ediff1d(nose_next_N_observ_y)
                nose_total_x_y_diff = (abs(sum(nose_consec_frames_x_diff)) /NOSE_ff_denom) \
                                    + (abs(sum(nose_consec_frames_y_diff)) /NOSE_ff_denom)   
                nose_avged_diff_indicator = 'avged_diff: ' + str(round(nose_total_x_y_diff,3))
                
                if counter in interpolated_NOSE:                                                                                   # if the value of nose is interpolated , will be different color
                    color = orange
                else :
                    color = yellow

                frame= WritetoFrame(frame,nose_x_axis_indicator,(1,nose_height),font,labelsize,color,1,cv2.LINE_4)
                frame= WritetoFrame(frame,nose_y_axis_indicator,(y_ax_coords,nose_height),font,labelsize,color,1,cv2.LINE_4)
                frame= WritetoFrame(frame,nose_l_axis_indicator,(likel_coords,nose_height),font,labelsize,color,1,cv2.LINE_4)
                frame= WritetoFrame(frame,nose_loc_diff_indicator,(loc_diff_coords,nose_height),font,labelsize,color,1,cv2.LINE_4)
                frame= WritetoFrame(frame,nose_avged_diff_indicator,(avg_diff_coords,nose_height),font,labelsize,color,1,cv2.LINE_4)
            
            # - - - - - - -Labels for second body part (if chosen by user)- - - - - - -
            if bdpart_opt != False and counter <= max_coord-15 :
                sec_x_axis_indicator = f'{bdpart_opt}     x: ' + str(round(coor[f'{bdpart_opt}x'][counter],3))
                sec_y_axis_indicator = 'y: ' + str(round(coor[f'{bdpart_opt}y'][counter],3))
                sec_l_axis_indicator = 'p: ' + str(round(coor[f'{bdpart_opt}likelihood'][counter],8))
                
                sec_curr_x_row, sec_curr_y_row  = coor[f'{bdpart_opt}x'][counter], coor[f'{bdpart_opt}y'][counter]                 #code to calculate the loc_diff
                sec_next_x_row, sec_next_y_row  = coor[f'{bdpart_opt}x'][counter+1], coor[f'{bdpart_opt}y'][counter+1]
                sec_total_diff_both_axes = abs (sec_curr_x_row-sec_next_x_row) + abs (sec_curr_y_row-sec_next_y_row)
                sec_loc_diff_indicator = 'loc_diff: ' + str(round(sec_total_diff_both_axes,3))
                
                sec_next_N_observ_x = coor[f'{bdpart_opt}x'].loc[counter+1:counter+ff_SECBD]
                sec_next_N_observ_y = coor[f'{bdpart_opt}y'].loc[counter+1:counter+ff_SECBD]                                                                       
                sec_consec_frames_x_diff, sec_consec_frames_y_diff = np.ediff1d \
                (sec_next_N_observ_x), np.ediff1d(sec_next_N_observ_y)
                sec_total_x_y_diff = (abs(sum(sec_consec_frames_x_diff)) /SECBD_ff_denom) \
                                   + (abs(sum(sec_consec_frames_y_diff)) /SECBD_ff_denom)
                sec_avged_diff_indicator = 'avged_diff: ' + str(round(sec_total_x_y_diff,3))
                
                if counter in interpolated_SECBD:                                                                                  # if the value of betwears is interpolated , will be different color
                    color = orange
                else :
                    color = yellow
                frame= WritetoFrame(frame,sec_x_axis_indicator,(1,sec_bd_height),font,labelsize,color,1,cv2.LINE_4)
                frame= WritetoFrame(frame,sec_y_axis_indicator,(y_ax_coords,sec_bd_height),font,labelsize,color,1,cv2.LINE_4)
                frame= WritetoFrame(frame,sec_l_axis_indicator,(likel_coords,sec_bd_height),font,labelsize,color,1,cv2.LINE_4)
                frame= WritetoFrame(frame,sec_loc_diff_indicator,(loc_diff_coords,sec_bd_height),font,labelsize,color,1,cv2.LINE_4)
                frame= WritetoFrame(frame,sec_avged_diff_indicator,(avg_diff_coords,sec_bd_height),font,labelsize,color,1,cv2.LINE_4)

            # - - - - - - -Labels for freezing (as dictated by freezing analysis output) - - - - - - -
            if counter in pkldata['all_first_finals']:
                frame= WritetoFrame(frame,'1st_freez',freezing_label_position,font,1,(0,0,255),2,cv2.LINE_4)
            if counter in pkldata['all_second_finals']:
                frame= WritetoFrame(frame,'2nd_freez',freezing_label_position_second_bdpart,font,1,(0,0,255),2,cv2.LINE_4)
            if counter < 50 :
                frame = WritetoFrame(frame,btw_msg,shrt_lbls_betw_positions,font,labelsize,yellow,1,cv2.LINE_4)                    # display how many times freezing was observed for betwears
                frame = WritetoFrame(frame,sec_msg,shrt_lbls_secbd_positions,font,labelsize,yellow,1,cv2.LINE_4)                   # display how many times freezing was observed for rightear


            output.write(frame)
                    
        else:
            break

    vid_capture.release()
    output.release()

In [15]:
def WritetoFrame (frame,indicator,position,font,size,color,thickness,linetype):
    """Inputs : same arguments as in the documentation of cv2.putText()
       Output : frame with an additional label each time this function is called
       
       
       This function is called by InsertLabels(). It adds one more label with information in a given frame."""
    
    cv2.putText(frame,indicator,position,font, size,color,thickness,linetype) 
    return frame
    

## Functions for checking below

In [16]:
def SelectedPartsCheck():
    "Stops the script if the UserSelections() returns None"
    if not selected_parts  :
        sys.exit(0)

In [17]:
def MatchPaths():
    """Uses the global variables pickle_directory and video_dir as defined by SelectDLCOutputDirectory and 
       SelectVideoDirectory, respectively. Finds matches between filenames and stops execution if the filename 
       of a pickle file does not correspond to any of the videos. Accepted video formats are mp4 and avi. This code can change.
       It is important that the name of the pickle file contains the characters 'DLC_' and only contains it once, so the function can 
       detect all the characters before the 'DLC_' and isolate the name of the file that was produced.
       
       Output : List with tuples as elements. Each tuple has str as elements. Examples :
        '446436_A02_20230516134908DLC_resnet50_Trigger_0005_008Aug30shuffle1_500000.csv.pickle', '446436_A02_20230516134908.mp4'

       """
    
    
#     pickle_directory
#     video_dir
    
    file_extension_check = False
    picklefiles    = [fi for fi in listdir(pickle_directory) if isfile(join(pickle_directory, fi))\
                         and fi[-7:]=='.pickle']                                                                                # isolate the pickle files only
    videofiles        = [vi for vi in listdir(video_dir) if isfile(join(video_dir,vi)) and vi[-3:] in ['mp4','avi']]
    videofiles_no_ext = [vino[:-4] for vino in videofiles]                                                                      #get rid of the .extension                                        
    double_verified   = []
    
    for pklfile in picklefiles :
        try :
            name_indx = pklfile.rfind("DLC_")                                                                                       #isolate filename before DLC_
            name_only = pklfile[:name_indx]
        except :
            print (f'Could not detect the sequence DLC_ in the name of {pklfile}. The file will be ignored !')
            continue
        if name_only in videofiles_no_ext :                                                                                     # search for the filename without extension in the list with video file names without extension
            videofil_index = videofiles_no_ext.index(name_only)                                                                 # the index of the file without extension will be used to trace the file with extension. This is possible because 'videofiles_no_ext' was created based on videofiles. Indices must be the same.   
            double_verified.append((pklfile,videofiles[videofil_index]))                                                        # not csv_name because we also want the extension. We want all information in this tuple
        else :
            print(f'\n\n Could not find a respective video file for {name_only}. This is an example of how filenames \
                  should look like : 446436_A02_20230516134908DLC_resnet50_Trigger_0005_008Aug30shuffle1_500000.csv.pickle\
                  and 446436_A02_20230516134908.mp4. Check if the files are named differently or \
                  if the extension of the video file is neither mp4 nor avi')
            
            
    return double_verified

## Main Code

#### Directories

In [18]:
# you can comment the next 3 lines and uncomment the first three lines right below them if you want user interface
pickle_directory = r'C:\Users\angdid\Desktop\EasyFreezy\Good_Quality_Env_3\Script1_output\pickle file for labelling (use in labelling script)'
video_dir = r'C:\Users\angdid\Desktop\EasyFreezy\Good_Quality_Env_3\rawvideos'
save_dir  = r'C:\Users\angdid\Desktop\TestEnv3'


#### Dimensions and ratios

In [23]:
vw              = 1600                                                                     # this is the video width 
vh              = 900                                                                      # this is the vido heigth
ul              = 'l'                                                                      # select whether you want your labels to be on the upper ('u') or lower part ('l') of the video
lbl_h_prop      = 10                                                                       # proportion of the video height within which the labels will be spread. Here it's only 10 percent, meaning 57 pixels range.
height_start    = 10                                                                       # although the range is from 10 to 57, the first pixels to be labelled will start from 10 so that they're visible
labelsize       = 0.5                                                                      # the size of your labels
short_labelsize = 0.5                                                                      # the size of temporary labels that display the overal occurences of freezing in the first second of the video

In [24]:
# SelectDLCOutputDirectory()
# SelectVideoDirectory()
# SelectSaveDirectory()
pkl_and_video_name:list = MatchPaths()                                                  # Example of a list with only one tuple [('446436_A02_20230516134908DLC_resnet50_Trigger_0005_008Aug30shuffle1_500000.csv.pickle', '446436_A02_20230516134908.mp4')]
iter_counter = -1                                                                       # will be used inside the main for loop to select the right tuple from pkl_and_video_name
selected_parts = UserSelections()                                                       # Selected body parts. betwears and nose is selected by default
SelectedPartsCheck()                                                                    # program will stop if no selections are returned 
bdpart_option = CheckIfSecondBodyPartIncluded()                                         # checks if user wants a 2nd bodypart coordinate


for both_names in pkl_and_video_name:
    pickle_dic = pd.read_pickle(join(pickle_directory,both_names[0]))
    pickle_df = pickle_dic[both_names[0][:-7]]                                         # dict has only one key which is the filename. -7 to get rid of .pickle
    RenameDataframeColumns()                                                           # make readable and easy to use columns in the csv key, which is the df with coordinates info
    print(f'\n Loading {both_names[0]} ')

    videoname = both_names[1]                              
    InsertLabels(videoname,pickle_df,bdpart_option) 
        




 Loading 1330_450418DLC_resnet50_SuccessfullProjectJul27shuffle1_500000.csv.pickle 

 file will be saved at C:\Users\angdid\Desktop\TestEnv3/1330_450418.avi
